# Electric Bill Parser Development

In [209]:
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Optional
import csv
import pdfplumber
import re
import os

@dataclass
class BillLineItem:
    bill_date: str
    category: str
    description: str
    quantity: Optional[float] = None
    unit: Optional[str] = None
    rate: Optional[float] = None
    amount: float = 0.0
    
    def to_csv_row(self):
        return [self.bill_date, self.category, self.description, 
                self.quantity or '', self.unit or '', self.rate or '', self.amount]

def extract_bill_date(pages):
    """Extract bill date from PDF text"""
    all_text = ' '.join(pages)
    # Try common date patterns
    patterns = [
        r'(?:\d{12}|\d{1}-\d{2}-\d{3}-\d{4}) (\d{2}/\d{2}/\d{2})'
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, all_text)
        if matches:
            return matches[0]
    
    # Fallback: extract from filename if available
    filename_match = re.search(r'(\d{4})_(\d{2})_(\d{2})', file)
    if filename_match:
        raise Exception("warning: we matched filename match instead of date pattern.  fix this code")
        year, month, day = filename_match.groups()
        return f"{month}/{day}/{year}"
    
    return "Unknown"

def parse_pages(pages):
    """Unified function to extract all line items from electric bill"""
    bill_date = extract_bill_date(pages)
    line_items = []
    all_text = ' '.join(pages)
    
    # Delivery usage charges (Time of Use)
    delivery_matches = re.findall(r'(Onpeak|Midpeak|Offpeak|Superoffpeak) (\d{1,3}(?:,\d{3})*)kWhx\$(\d+\.?\d+) \$(\d+\.\d+)', all_text)
    for tou, kwh, rate, amount in delivery_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Delivery",
            description=f"{tou} Usage",
            quantity=float(kwh.replace(',', '')),
            unit="kWh",
            rate=float(rate),
            amount=float(amount)
        ))

    # Delivery usage charges (Tiered)
    delivery_matches = re.findall(r'(Tier\d.*) (\d+(?:,\d{3})*)kWhx\$(\d\.\d+) \$(\d+.\d+)', all_text)
    for tier, kwh, rate, amount in delivery_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Delivery",
            description=f"{tier} Usage",
            quantity=float(kwh.replace(',', '')),
            unit="kWh",
            rate=float(rate),
            amount=float(amount)
        ))
    
    
    # Basic charge
    basic_matches = re.findall(r'(Basiccharge) (\d+)daysx\$(\d\.?\d+) \$(\d.?\d+)', all_text)
    for _, days, rate, amount in basic_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Delivery Surcharge",
            description="Basic Charge",
            quantity=float(days),
            unit="days",
            rate=float(rate),
            amount=float(amount)
        ))
    
    # Delivery surcharges
    surcharge_matches = re.findall(r'(Baselinecredit|PCIA|CCAwildfirefundcharge|CCAwildfirecharge|CTC|Fixedrecoverycharge|DWR.*) (\d{1,3}(?:,\d{3})*)kWhx(-?\$\d.?\d+) (-?\$\d+.?\d+)', all_text)
    for charge_type, kwh, rate, amount in surcharge_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Delivery Surcharge",
            description=charge_type.replace('charge', ' charge').replace('credit', ' credit'),
            quantity=float(kwh.replace(',', '')),
            unit="kWh",
            rate=float(rate.replace('$', '')),
            amount=float(amount.replace('$', ''))
        ))

    # Generation municipal surcharge
    municipal_matches = re.findall(r'(GenerationMunicipalSurcharge) \$(\d+.?\d+)', all_text)
    for charge_type, amount in municipal_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Generation Surcharge",
            description="Municipal Surcharge",
            amount=float(amount)
        ))
         
    # Generation CA Climate Credit
    municipal_matches = re.findall(r'(CAClimateCredit) (-\$\d+.?\d+)', all_text)
    for charge_type, amount in municipal_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Generation Surcharge",
            description="CA Climate Credit",
            amount=float(amount.replace('$', ''))
        ))

    # Generation usage charges
    gen_matches = re.findall(r'100%GreenPower-(.*) (\d+.?\d+)kWh@(\d+.\d+) \$(\d+.\d+)', all_text)    
    for tou, kwh, rate, amount in gen_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Generation",
            description=f"Green Power {tou}",
            quantity=float(kwh),
            unit="kWh",
            rate=float(rate),
            amount=float(amount)
        ))

    # Generation usage charge - total
    gen_matches = re.findall(r'CleanPower-(.*)(\d+\.?\d*)kWh@(\d\.\d+) \$(\d+.\d+)', all_text) 
    for tou, kwh, rate, amount in gen_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Generation",
            description=f"{tou}",
            quantity=float(kwh),
            unit="kWh",
            rate=float(rate),
            amount=float(amount)
        ))
         
    # Energy surcharge
    energy_matches = re.findall(r'(EnergySurcharge) \$(\d+.?\d+)', all_text)
    for charge_type, amount in energy_matches:
        line_items.append(BillLineItem(
            bill_date=bill_date,
            category="Generation Surcharge",
            description="Energy Surcharge",
            amount=float(amount)
        ))
         
    return line_items
         

def get_pages(file):
    pages = []
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            pages.append(text)

    return pages

def parse_file(file):
    pages = get_pages(file)
    line_items = parse_pages(pages)
    return line_items

def write_to_csv(line_items, filename="electric_bill.csv"):
    headers = ['bill_date', 'category', 'description', 'quantity', 'unit', 'rate', 'amount']
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        for item in line_items:
            writer.writerow(item.to_csv_row())
    
def parse_files(files, dest_dir):
    actual_totals = {}
    all_line_items = []
    for src_file in files:
        dest_file = Path(dest_dir) / Path(Path(src_file).stem).with_suffix('.csv')
        line_items = parse_file(src_file)
        all_line_items.extend(line_items)
        write_to_csv(line_items, dest_file)
        amount = f'{sum(item.amount for item in line_items):.2f}'
        bill_date = Path(src_file).stem
        actual_totals[bill_date] = amount
        print(f"Wrote {dest_file}, {len(line_items)} lines, Total amount: ${amount}")
    dest_file = Path(dest_dir) / Path("electric_bills.csv")
    write_to_csv(all_line_items, dest_file)
    print(f"Wrote {dest_file}, {len(all_line_items)} lines, Total amount: ${sum(item.amount for item in all_line_items):.2f}")
    return actual_totals

def test_totals(expected, actual):
    for k, v in actual.items():
        assert expected[k] == v, f"For {k} expected {expected[k]} but got {v}"

In [210]:
EXPECTED_TOTALS = {
    '2020_04_21': '211.57',
    '2020_05_21': '246.25',
    '2020_06_20': '203.83',
    '2020_07_23': '250.72',
    '2020_08_21': '236.38',
    '2020_09_22': '370.68',
    '2020_10_22': '507.53',
    '2020_11_21': '285.58',
    '2020_12_22': '368.60',
    '2021_01_23': '315.62',
    '2021_02_23': '354.05',
    '2021_03_24': '379.87',
    '2021_04_13': '317.13',
    '2021_05_10': '284.96',
    '2021_06_08': '291.59',
    '2021_07_09': '304.25',
    '2021_08_09': '299.94',
    '2021_09_07': '289.26',
    '2021_10_08': '292.28',
    '2021_11_08': '327.21',
    '2021_12_09': '378.01',    
    '2022_01_07': '418.32',
    '2022_02_07': '370.86',
    '2022_03_09': '382.32',
    '2022_04_07': '235.60',
    '2022_05_09': '289.90',
    '2022_06_08': '308.33',
    '2022_07_11': '307.23',
    '2022_08_08': '340.93',
    '2022_09_07': '406.74',
    '2022_10_08': '622.93',
    '2022_11_07': '409.07',
    '2022_12_08': '477.72',    
    '2023_01_09': '589.27',
    '2023_02_07': '755.97',
    '2023_03_08': '647.39',
    '2023_04_06': '596.49',
    '2023_05_08': '585.54',
    '2023_06_07': '460.55',
    '2023_07_07': '508.09',
    '2023_08_07': '487.45',
    '2023_09_06': '626.12',
    '2023_10_05': '530.13',
    '2023_11_03': '519.29',
    '2023_12_06': '649.19',    
    '2024_01_08': '670.57',
    '2024_02_06': '700.36',
    '2024_03_08': '759.51',
    '2024_04_08': '597.39',
    '2024_05_07': '714.01',
    '2024_06_06': '613.85',
    '2024_07_08': '565.10',
    '2024_08_07': '654.61',
    '2024_09_06': '739.13',
    '2024_10_07': '668.68',
    '2024_11_05': '554.15',
    '2024_12_06': '776.72',
    '2025_01_07': '774.89',
    '2025_02_05': '969.10',
    '2025_03_07': '894.63',
    '2025_04_07': '754.42',
    '2025_05_07': '808.63',
    '2025_06_06': '630.18',
    '2025_07_09': '627.18',
}

dest_dir = "/Users/ericmelz/Data/Bills/Electric/CSV"
#years = ['2024', '2025']
years = ['2020']
files = []

for year in years:
    src_dir = Path("/Users/ericmelz/Data/Bills/Electric/PDF") / year
    for path in src_dir.iterdir():        
        if path.suffix.lower() == ".pdf":
            files.append(path)


In [211]:
actual_totals = parse_files(files, dest_dir)

Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_11_21.csv, 9 lines, Total amount: $285.58
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_07_23.csv, 10 lines, Total amount: $250.72
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_09_22.csv, 9 lines, Total amount: $370.68
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_05_21.csv, 13 lines, Total amount: $246.25
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_12_22.csv, 9 lines, Total amount: $368.60
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_10_22.csv, 12 lines, Total amount: $507.53
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_06_20.csv, 13 lines, Total amount: $203.83
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_04_21.csv, 10 lines, Total amount: $211.57
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/2020_08_21.csv, 9 lines, Total amount: $236.38
Wrote /Users/ericmelz/Data/Bills/Electric/CSV/electric_bills.csv, 94 lines, Total amount: $2681.14


In [212]:
test_totals(EXPECTED_TOTALS, actual_totals)

# Scratchpad for testing

In [200]:
pages = get_pages(files[2])

In [204]:
page2 = pages[2]
page3 = pages[3]
page4 = pages[4]
page5 = pages[5]

In [205]:
page2

"MELZ,ERIC/ Page3of8\nServiceaccount 3-047-6370-74 DELIVERY\nServiceaddress 300SREEVESDR\nSOUTHERN CALIFORNIA EDISON\nBEVERLYHILLS,CA90212\ndelivers your electricity\nRotatingoutage GroupA007\nYour past and current electricity usage\nFormeter223000-016505from08/05/20to09/03/20 Yournextbillingcyclewillendonorabout10/06/20.\nTotalelectricityyouusedthismonthinkWh 1,488\nYourdailyaverageelectricityusage(kWh)\n2Yearsago:79.84 Lastyear:44.00 Thisyear:51.31\n80\n53\n27\n0\nAug Aug Sep Oct Nov Dec Jan Feb Mar Apr May Jun Jul Aug\n'18 '19 '19 '19 '19 '19 '20 '20 '20 '20 '20 '20 '20 '20\nDetails of your new charges\nYourrate:DOMESTIC(SCE)\nBillingperiod:08/05/20to09/03/20(29days)\nDeliverycharges -Costtodeliveryourelectricity\nYourDeliverychargesinclude:\nBasiccharge 29daysx$0.03100 $0.90\n$27.94transmissioncharges\nEnergy-Summer\n·$133.38distributioncharges\nTier1(100%ofbaseline) 479kWhx$0.10613 $50.84\n·-$0.74nucleardecommissioning\nTier2(101%to400%) 1,009kWhx$0.16542 $166.91\n·charges\n$18.84

In [206]:
re.findall(r'(Tier\d.*) (\d+)kWhx\$(\d\.\d+) \$(\d+.\d+)', 'Tier1(100%ofbaseline) 479kWhx$0.10613 $50.84\n·-$0.74nucleardecommissioning\nTier2(101%to400%) 1,009kWhx$0.16542 $166.91\n')

[('Tier1(100%ofbaseline)', '479', '0.10613', '50.84')]

In [208]:
re.findall(r'(Tier\d.*) (\d+(?:,\d{3})*)kWhx', 'Tier1(100%ofbaseline) 479kWhx$0.10613 $50.84\n·-$0.74nucleardecommissioning\nTier2(101%to400%) 1,009kWhx$0.16542 $166.91\n')

[('Tier1(100%ofbaseline)', '479'), ('Tier2(101%to400%)', '1,009')]

In [170]:
re.findall(r'(CCAwildfirecharge 1,013kWhx\$0.00652 \$6.6)', 'CCAwildfirecharge 1,013kWhx$0.00652 $6.6')

['CCAwildfirecharge 1,013kWhx$0.00652 $6.6']

In [172]:
re.findall(r'(CCAwildfirecharge 1,013kWhx\$0.00652 \$6.6)', page4)

['CCAwildfirecharge 1,013kWhx$0.00652 $6.6']

In [173]:
re.findall(r'(Baselinecredit|PCIA|CCAwildfirefundcharge|CTC|Fixedrecoverycharge|DWR.*) (\d{1,3}(?:,\d{3})*)kWhx(-?\$\d.?\d+) (-?\$\d+.?\d+)', page4)

[('Baselinecredit', '495', '-$0.08798', '-$43.55'),
 ('PCIA', '1,013', '$0.00371', '$3.76'),
 ('DWRadjustment', '1,013', '-$0.00208', '-$2.11'),
 ('CTC', '1,013', '-$0.00020', '-$0.20'),
 ('Fixedrecoverycharge', '1,013', '$0.00096', '$0.97')]

In [174]:
re.findall(r'(Baselinecredit|PCIA|CCAwildfirefundcharge|CTC|Fixedrecoverycharge|DWR.*) (.*)', page4)

[('Baselinecredit', '495kWhx-$0.08798 -$43.55'),
 ('PCIA', '1,013kWhx$0.00371 $3.76'),
 ('DWRadjustment 1,013kWhx-$0.00208', '-$2.11'),
 ('CTC', '1,013kWhx-$0.00020 -$0.20'),
 ('Fixedrecoverycharge', '1,013kWhx$0.00096 $0.97')]

In [178]:
re.findall(r'(CCAwildfirefundcharge)', 'CCAwildfirecharge')

[]

In [127]:
re.findall(r'(CleanPower-Total) (1265.2)kWh@(0.0846) \$107.04', 'CleanPower-Total 1265.2kWh@0.0846 $107.04') 

[('CleanPower-Total', '1265.2', '0.0846')]

In [128]:
re.findall(r'(CleanPower-Total) (\d+\.?\d*)kWh@(\d\.\d+) \$107.04', 'CleanPower-Total 1265.2kWh@0.0846 $107.04') 

[('CleanPower-Total', '1265.2', '0.0846')]

In [129]:
re.findall(r'(CleanPower-Total) (\d+\.?\d*)kWh@(\d\.\d+) \$\d+.\d+', 'CleanPower-Total 1265.2kWh@0.0846 $107.04') 

[('CleanPower-Total', '1265.2', '0.0846')]

In [130]:
re.findall(r'(CleanPower-Total) (\d+\.?\d*)kWh@(\d\.\d+) \$\d+.\d+', page5) 

[('CleanPower-Total', '1265.2', '0.0846')]